In [3]:
import pandas as pd
import mysql.connector
from mysql.connector import Error

# Step 1: Read the Excel file
data_file = r"C:\Users\CHAKU FOODS\Documents\Final Tree Photo Taking Cleaning\Tree Data.xlsx"
data = pd.read_excel(data_file)

# Step 2: Establish connection to the MySQL database
try:
    connection = mysql.connector.connect(
        host='localhost',  # Change to your host
        user='root',  # Replace with your MySQL username
        password='KpakpoParker700!',  # Replace with your MySQL password
        database='chaku_foods'  # Replace with your database name
    )

    if connection.is_connected():
        cursor = connection.cursor(dictionary=True)

        # Step 3: Retrieve farmer and farm IDs based on last_name, first_name, and farm_number
        for index, row in data.iterrows():
            last_name = row['last_name']
            first_name = row['first_name']
            farm_number = row['farm_number']

            # Query to get farmer_id
            cursor.execute(
                """
                SELECT farmer_id FROM farmer WHERE last_name = %s AND first_name = %s
                """, (last_name, first_name)
            )
            farmer_result = cursor.fetchone()

            if farmer_result:
                farmer_id = farmer_result['farmer_id']

                # Ensure all previous results are read before executing a new query
                cursor.fetchall()

                # Query to get farm_id
                cursor.execute(
                    """
                    SELECT farm_id FROM farm WHERE farm_number = %s
                    """, (farm_number,)
                )
                farm_result = cursor.fetchone()

                if farm_result:
                    farm_id = farm_result['farm_id']

                    # Ensure all previous results are read before executing a new query
                    cursor.fetchall()

                    # Step 4: Prepare the insert statement
                    insert_query = """
                    INSERT INTO tree_data (
                        farmer_id, farm_id, tree_number, picture_1_url, picture_2_url, picture_3_url, 
                        picture_4_url, picture_5_url, picture_6_url, picture_7_url, picture_8_url, 
                        fruit_set_percentage
                    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                    """

                    cursor.execute(
                        insert_query,
                        (
                            farmer_id,
                            farm_id,
                            row['tree_number'] if pd.notna(row['tree_number']) else None,
                            row['picture_1_url'] if pd.notna(row['picture_1_url']) else None,
                            row['picture_2_url'] if pd.notna(row['picture_2_url']) else None,
                            row['picture_3_url'] if pd.notna(row['picture_3_url']) else None,
                            row['picture_4_url'] if pd.notna(row['picture_4_url']) else None,
                            row['picture_5_url'] if pd.notna(row['picture_5_url']) else None,
                            row['picture_6_url'] if pd.notna(row['picture_6_url']) else None,
                            row['picture_7_url'] if pd.notna(row['picture_7_url']) else None,
                            row['picture_8_url'] if pd.notna(row['picture_8_url']) else None,
                            row['fruit_set_percentage'] if pd.notna(row['fruit_set_percentage']) else None
                        )
                    )

        # Step 5: Commit the transaction
        connection.commit()
        print("Data imported successfully!")

except Error as e:
    print(f"Error: {e}")

finally:
    # Step 6: Close the connection
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection closed.")


Data imported successfully!
MySQL connection closed.
